# Instacart Recommender PoC
## Introduction

### Purpose of this notebook
The purpose of this notebook is to assess the validity of using an item-based recommender for Instacart

### Prior work
* EDA

### Next Steps
* Build recommender on InstaCart purchases

### Guiding questions
* What is a good proxy for rating? 
* Is order data, rather than user data, an OK granularity level? 

### Key findings
* INCLUDE HERE

### Related Notebooks
* Insert EDA Notebook here... 

## Imports and configurations

In [1]:
import pandas as pd
import numpy as np
from surprise import KNNBaseline, Dataset, Reader, accuracy
import matplotlib.pyplot as plt
import psycopg2
import os
import matplotlib.pyplot as plt
from surprise.model_selection import train_test_split

## Load and prep data

Read in CSVs from Instacart.

In [2]:
orders = pd.read_csv("../../Data/order_products__train.csv")
products = pd.read_csv("../../Data/products.csv")
aisles = pd.read_csv("../../Data/aisles.csv")

In [12]:
def format_recommend(orders):
  """ This function takes the InstaCart orders dataframe 
  and returns formats the dataframe for the surprise 
  recommendation library of the top 250 products sold by 
  InstaCart for model and app simplicity. 

  Args: 
    orders(csv): Instacart order_products__train.csv

  Returns:
    Dataframe with columns order_id, product_id, and rating

  """
  counts = orders['product_id'].value_counts().reset_index().head(250)
  counts.columns = ['product_id', 'frequency_count']
  orders = orders[orders.product_id.isin(counts.product_id)]
  orders['rating'] = np.log(orders.add_to_cart_order)
  orders = orders[['order_id', 'product_id','rating']]
  return(orders)

In [13]:
recommend = format_recommend(orders)
recommend.shape

/Users/sarahgreenwood/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(450859, 3)

## Build and test model

Convert data to surprise format and set scale as the minimum being the best and the max rating being the worst. 

In [15]:
def get_data(data):
  reader = Reader(rating_scale = (max(data.rating),min(data.rating)))
  data = Dataset.load_from_df(data, reader)
  return(data)

In [16]:
recommend = get_data(recommend)

In [18]:
trainset, testset = train_test_split(recommend, test_size=.25)

In [19]:
sim_options = {
    'name': 'pearson_baseline', 
    'user_based': False}
algo = KNNBaseline(k=10, sim_options = sim_options)
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [20]:
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 2.8038


2.8037817084012833